In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using BenchmarkTools
using JLD2

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [88]:
function normalize_centers(centers, radii)
    X = reshape(centers, (3,Int(length(centers)/3)))
    com = mean(eachcol(X))
    X = (X .- com)'
    (_,_,V) = svd(X, full = true)
    X = (X*V)'
    reshape(X, (1, size(X)[1] * size(X)[2]))
end

normalize_centers (generic function with 1 method)

In [95]:
file = "assets/input/poly/2_6sag_protor_hardsphere_assembly.poly"
n_mol = 2
flat_centers, radii = MorphoMol.Utilities.poly_to_state(file)
flat_centers = normalize_centers(flat_centers, radii)

n = Int(length(flat_centers)/3/n_mol)
m1 = flat_centers[1:n*3]
m2 = flat_centers[n*3+1:end]
template_radii = radii[1:n];

X = reshape(m1, (3, n))
Y = reshape(m2, (3, n))

com1 = mean(eachcol(X))
println(com1)

X = X .- com1
Y = Y .- com1

com2 = mean(eachcol(Y))
Y = Y .- com2
Y = transpose(Y)

U,S,V = svd(X*Y)
R = RotMatrix{3}(V*transpose(U))
T = com2

[-6.110276929059617, -11.346446194348704, -0.0044799749439093746]


3-element Vector{Float64}:
 12.220553858119201
 22.692892388697537
  0.008959949887908238

In [106]:
R_gen = log(R)
x_init = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, R_gen[3,2], R_gen[1,3], R_gen[2,1], T[1], T[2], T[3]]
template_mol = reshape(X, (1, size(X)[1] * size(X)[2]))

1×3618 Matrix{Float64}:
 28.739  -3.13357  6.51482  27.9039  …  3.81332  32.1299  -5.94  2.04325

In [107]:
jldsave("assets/input/jld2/2_6sag_protor_hardsphere_assembly.jld2"; template_mol, template_radii, x_init)

In [108]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [109]:
@load "assets/input/jld2/2_6sag_protor_hardsphere_assembly.jld2" template_mol template_radii x_init
n_atoms_per_mol = length(template_mol) ÷ 3
n_mol = length(x_init) ÷ 6
template_mol = reshape(template_mol,(3,n_atoms_per_mol))
radii = vcat([template_radii for i in 1:n_mol]...);
MorphoMol.Utilities.state_to_poly(get_flat_realization(x_init, template_mol), radii, "assets/output/start", n_mol, n_atoms_per_mol)